In [1]:
import parsl
from parsl.config import Config
from parsl.executors import HighThroughputExecutor
from parsl.launchers import SrunLauncher
from parsl.providers import SlurmProvider
from parsl.dataflow.futures import AppFuture as Future

# Notes:
# - Need: `ntasks=<number>`; Has: `ntasks-per-node=<number>`
# - Remove: `exclusive`

account = "brlab"

config = Config(
    executors=[
        HighThroughputExecutor(
            label="quacc_parsl",
            provider=SlurmProvider(
                account=account,
                partition="batch",
                worker_init=f'eval "$(pixi shell-hook --manifest-path=/home/avcopan/code/workflow-practice/pixi.toml)"',
                walltime="00:30:00",
                scheduler_options='#SBATCH --ntasks=1',
                mem_per_node=4,
                launcher=SrunLauncher(),
                exclusive=False,
            ),
        )
    ],
)

parsl.load(config)

/home/avcopan/code/workflow-practice/.pixi/envs/default/lib/python3.11/site-packages/paramiko/pkey.py:100: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "cipher": algorithms.TripleDES,
/home/avcopan/code/workflow-practice/.pixi/envs/default/lib/python3.11/site-packages/paramiko/transport.py:259: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "class": algorithms.TripleDES,
/home/avcopan/code/workflow-practice/.pixi/envs/default/lib/python3.11/site-packages/paramiko/pkey.py:100: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "cipher": algorithms.TripleDES,
/home/avcopan/code/workflow-practice/.pixi/envs/default/lib/python3.11/site

In [2]:
import quacc


@quacc.job
def slow_job(arg):
    import subprocess
    subprocess.run(["sleep", "5"])
    return arg + 1


@quacc.flow
def workflow(arg):
    return [slow_job(arg) for _ in range(4)]

In [3]:
futures: list[Future] = workflow(9)
results = [f.result() for f in futures]
print(results)  # 9

[10, 10, 10, 10]


In [4]:
futures: list[Future] = workflow(11)
results = [f.result() for f in futures]
print(results)  # 9

[12, 12, 12, 12]


In [5]:
futures: list[Future] = workflow(18)
results = [f.result() for f in futures]
print(results)  # 9

[19, 19, 19, 19]
